# Agenda
1. Introduction
2. Business Questions
3. Business Assumptions
4. Solution Planning
5. Importing and Cleaning Data
6. Exploratory Data Analysis
   1. Hyphotesis Testing
   2. Answering Business Questions
7. Solutions
8.  Conclusion
9.  Next Steps

# 1) Introduction

This is a insight project designed to help an fictional CEO of a house sales company, located at King Count (USA). My aim is to draw thoughtful conclusions based on the houses dataset and business questions that he provided, validating or not business hyphotesis and generating maps and tables that can be used to assist him to take decisions based on data.

# 2) Business Questions

**We have two main questions:**
1. Which are the real states that House Rocket should buy, and how much will it cost?
2. What is the best moment to sell the real states, and how much will it cost?

# 3) Business Assumptions

+ We assumed that the season and region have an impact on the houses prices.
+ We considered old houses as the ones built before 1955.

# 4) Solution Planning

#### 4.1. Which are the real states that House Rocket should buy, and how much will it cost?

**Steps**
1. Collect the data on Kaggle
2. Group by region (zipcode)
3. For each region we should find the median price
4. Houses whose price is less than the median price, and is in good condition ($\geq 2$) should be bought by the suggested price
5. The final table to delivered will have the following structure:

|ID    | Region |Price | Median Price | Condition | Status |
|------|--------|-----------------|------------------|----------|--------|
|10330 |302349  |U\$ 450.000        |U\$ 500.000        |3         |Buy  |

#### 4.2. What is the best moment to sell the real states, and how much will it cost?

**Steps**
1. Group by region (zipcode) and by season
2. For each region + season group, we calculate the mean price
3. Sale conditions:
    + If the house price is bigger than the median of region + season
        + The sale price will be the purchase price + 10 %
    
    + If the house price is lower than the median of region + season
        +  The sale price will be the purchase price + 30 %
    
4. The final table to delivered will have the following structure:

|ID | Region | Season | Median Price | Purchase Price | Sale Price |Lucro| Condition |
|----------| ----------|----------|----------|---------- |----------|----------| ---------- |
|10330 | 302349 | Verão | U\$ 800.000,00 | U\$ 450.000,00 | U\$ 450.000,00 + 30\%| ??|

# 5) Importing and Cleaning Data

**Importing Libraries**

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

%matplotlib inline
pd.set_option('display.float_format', '{:.2f}'.format)

**Importing data**

In [29]:
house_rocket = pd.read_csv( 'kc_house_data.csv' )
house_rocket.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00,3,1.00,1180,5650,1.00,0,0,...,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,20141209T000000,538000.00,3,2.25,2570,7242,2.00,0,0,...,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,20150225T000000,180000.00,2,1.00,770,10000,1.00,0,0,...,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,20141209T000000,604000.00,4,3.00,1960,5000,1.00,0,0,...,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,20150218T000000,510000.00,3,2.00,1680,8080,1.00,0,0,...,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503


**Displaying summary statistics**

In [30]:
house_rocket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [31]:
house_rocket.describe().round()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00
mean,4580301521.00,540088.00,3.00,2.00,2080.00,15107.00,1.00,0.00,0.00,3.00,8.00,1788.00,292.00,1971.00,84.00,98078.00,48.00,-122.00,1987.00,12768.00
std,2876565571.00,367127.00,1.00,1.00,918.00,41421.00,1.00,0.00,1.00,1.00,1.00,828.00,443.00,29.00,402.00,54.00,0.00,0.00,685.00,27304.00
min,1000102.00,75000.00,0.00,0.00,290.00,520.00,1.00,0.00,0.00,1.00,1.00,290.00,0.00,1900.00,0.00,98001.00,47.00,-123.00,399.00,651.00
25%,2123049194.00,321950.00,3.00,2.00,1427.00,5040.00,1.00,0.00,0.00,3.00,7.00,1190.00,0.00,1951.00,0.00,98033.00,47.00,-122.00,1490.00,5100.00
50%,3904930410.00,450000.00,3.00,2.00,1910.00,7618.00,2.00,0.00,0.00,3.00,7.00,1560.00,0.00,1975.00,0.00,98065.00,48.00,-122.00,1840.00,7620.00
75%,7308900445.00,645000.00,4.00,2.00,2550.00,10688.00,2.00,0.00,0.00,4.00,8.00,2210.00,560.00,1997.00,0.00,98118.00,48.00,-122.00,2360.00,10083.00
max,9900000190.00,7700000.00,33.00,8.00,13540.00,1651359.00,4.00,1.00,4.00,5.00,13.00,9410.00,4820.00,2015.00,2015.00,98199.00,48.00,-121.00,6210.00,871200.00


In [32]:
house_rocket.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.00,-0.02,0.00,0.01,-0.01,-0.13,0.02,-0.00,0.01,-0.02,0.01,-0.01,-0.01,0.02,-0.02,-0.01,-0.00,0.02,-0.00,-0.14
price,-0.02,1.00,0.31,0.53,0.70,0.09,0.26,0.27,0.40,0.04,0.67,0.61,0.32,0.05,0.13,-0.05,0.31,0.02,0.59,0.08
bedrooms,0.00,0.31,1.00,0.52,0.58,0.03,0.18,-0.01,0.08,0.03,0.36,0.48,0.30,0.15,0.02,-0.15,-0.01,0.13,0.39,0.03
bathrooms,0.01,0.53,0.52,1.00,0.75,0.09,0.50,0.06,0.19,-0.12,0.66,0.69,0.28,0.51,0.05,-0.20,0.02,0.22,0.57,0.09
sqft_living,-0.01,0.70,0.58,0.75,1.00,0.17,0.35,0.10,0.28,-0.06,0.76,0.88,0.44,0.32,0.06,-0.20,0.05,0.24,0.76,0.18
sqft_lot,-0.13,0.09,0.03,0.09,0.17,1.00,-0.01,0.02,0.07,-0.01,0.11,0.18,0.02,0.05,0.01,-0.13,-0.09,0.23,0.14,0.72
floors,0.02,0.26,0.18,0.50,0.35,-0.01,1.00,0.02,0.03,-0.26,0.46,0.52,-0.25,0.49,0.01,-0.06,0.05,0.13,0.28,-0.01
waterfront,-0.00,0.27,-0.01,0.06,0.10,0.02,0.02,1.00,0.40,0.02,0.08,0.07,0.08,-0.03,0.09,0.03,-0.01,-0.04,0.09,0.03
view,0.01,0.40,0.08,0.19,0.28,0.07,0.03,0.40,1.00,0.05,0.25,0.17,0.28,-0.05,0.10,0.08,0.01,-0.08,0.28,0.07
condition,-0.02,0.04,0.03,-0.12,-0.06,-0.01,-0.26,0.02,0.05,1.00,-0.14,-0.16,0.17,-0.36,-0.06,0.00,-0.01,-0.11,-0.09,-0.00


### 5.1 Transforming Data

In [33]:
# Converting 'date' column from object to datetime
house_rocket["date"] = pd.to_datetime( house_rocket["date"] )

In [34]:
# Adding new columns
house_rocket["year"] = house_rocket["date"].dt.year
house_rocket["month"] = house_rocket["date"].dt.month

print( house_rocket["year"].head(3) )
print( house_rocket["month"].head(3) )

0    2014
1    2014
2    2015
Name: year, dtype: int64
0    10
1    12
2     2
Name: month, dtype: int64


In [35]:
# Creating season column
# house_rocket["season"] = house_rocket["date"].apply( lambda x: 'Spring' if ((x >= pd.to_datetime('2014-03-20')) & (x < pd.to_datetime('2014-06-21'))) | ((x >= pd.to_datetime('2015-03-20')) & (x < pd.to_datetime('2015-06-20'))) else \
#                                           'Summer' if (x >= pd.to_datetime('2014-06-21')) & (x < pd.to_datetime('2014-09-22'))  else \
#                                           'Fall' if (x >= pd.to_datetime('2014-09-22')) & (x < pd.to_datetime('2014-12-21')) else \
#                                           'Winter' )
# house_rocket["season"].unique()

In [36]:
house_rocket["season"] = house_rocket.apply(lambda x: 'Spring' if ((x['date'] >= pd.to_datetime('2014-03-20')) & (x['date'] < pd.to_datetime('2014-06-21'))) | ((x['date'] >= pd.to_datetime('2015-03-20')) & (x['date'] < pd.to_datetime('2015-06-20'))) else
                                          'Summer' if (x['date'] >= pd.to_datetime('2014-06-21')) & (x['date'] < pd.to_datetime('2014-09-22'))  else
                                          'Fall' if (x['date'] >= pd.to_datetime('2014-09-22')) & (x['date'] < pd.to_datetime('2014-12-21')) else
                                          'Winter', axis=1)
house_rocket["season"].unique()

array(['Fall', 'Winter', 'Spring', 'Summer'], dtype=object)

### 5.2 Cleaning Data

**Checking for null values**

In [37]:
house_rocket.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
year             0
month            0
season           0
dtype: int64

**Checking for duplicated lines**

In [38]:
house_rocket.duplicated().sum()

0

# 6) Exploratory Data Analysis

## 6.1. Hyphotesis Testing

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

In [39]:
m_price_with_waterfront = house_rocket.loc[house_rocket["waterfront"] == 1].price.mean()
m_price_all = house_rocket.price.mean()

print( f'Média dos preço das casas com vista para água é de {m_price_with_waterfront:,.2f} dólares.' )
print( f'Média dos preço de todas as casas é de {m_price_all:,.2f} dólares.' )

Média dos preço das casas com vista para água é de 1,661,876.02 dólares.
Média dos preço de todas as casas é de 540,088.14 dólares.


In [40]:
porc_aumento_price = (m_price_with_waterfront / m_price_all * 100) - 100
print( f'Imóveis que possuem vista para água são {porc_aumento_price:.1f}% mais caros, na média.' )

Imóveis que possuem vista para água são 207.7% mais caros, na média.


#### **H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média..**

In [41]:
mean_price_1955 = house_rocket.loc[house_rocket["yr_built"] < 1955].price.mean()

print( f'Imóveis com data de construção menor que 1955 tem um preço médio de {mean_price_1955:,.2f} dólares.')

Imóveis com data de construção menor que 1955 tem um preço médio de 537,050.91 dólares.


In [42]:
porc_dec_1955 = 100 - ( mean_price_1955 / m_price_all * 100 )
print( f'Imóveis que possuem vista para água são {porc_dec_1955:.1f}% mais baratos, na média.' )

Imóveis que possuem vista para água são 0.6% mais baratos, na média.


#### **H3: Imóveis sem porão possuem área total(sqrt_lot), são 40% maiores do que os imóveis com porão.**

In [43]:
m_size_without_b = house_rocket.loc[house_rocket["sqft_basement"] == 0].sqft_lot.mean()
m_size_with_b = house_rocket.loc[house_rocket["sqft_basement"] > 0].sqft_lot.mean()
porc_increase_size = ( m_size_without_b / m_size_with_b * 100 ) - 100

print( f'Casas sem porão são {porc_increase_size:.2f}% maiores na média do que casas com porão.' )

Casas sem porão são 22.56% maiores na média do que casas com porão.


#### **H4: O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%.**

In [44]:
house_rocket.loc[:, ['price', 'year']].groupby( 'year' ).mean().reset_index()

,year,price
0,2014,539181.43
1,2015,541988.99


In [45]:
m_price_2014 = house_rocket.loc[house_rocket["year"] == 2014].price.mean()
m_price_2015 = house_rocket.loc[house_rocket["year"] == 2015].price.mean()
# porc_increse_yoy = 100 * (m_price_2015 - m_price_2014) / m_price_2015
porc_increse_yoy = 100 - (m_price_2014 / m_price_2015 * 100)

print( f'O cresimento YoY foi de apenas {porc_increse_yoy:.2f}% entre os anos 2014 e 2015.' )

O cresimento YoY foi de apenas 0.52% entre os anos 2014 e 2015.


In [46]:
# Standard deviation
std_price_2014 = house_rocket[house_rocket['year']==2014]['price'].std()
std_price_2015 = house_rocket[house_rocket['year']==2015]['price'].std()

In [47]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='2014',
    x=['Year'], y=[mean_price_2014],
    error_y=dict(type='data', array=[std_price_2014])
))
fig.add_trace(go.Bar(
    name='2015',
    x=['Year'], y=[mean_price_2015],
    error_y=dict(type='data', array=[std_price_2015])
))
fig.update_layout(barmode='group')
fig.show()

NameError: name 'mean_price_2014' is not defined

#### **H5: Imóveis com 3 banheiros tem um crescimento de MoM ( Month over Month ) de 15%.**

In [ ]:
df_h5 = house_rocket.loc[house_rocket["bathrooms"] == 3, ['date', 'month', 'price', 'bathrooms']]
df_h5_m_prices = df_h5.loc[df_h5["bathrooms"] == 3].groupby( 'month' ).price.agg(['mean']).T

df_h5_cols = {1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"}

df_h5_m_prices.rename( df_h5_cols, axis=1, inplace=True )
df_h5_m_prices.head()

month,January,February,March,April,May,June,July,August,September,October,November,December
mean,672923.170732,696714.422222,768495.096154,709751.988235,689252.1,775057.564103,741405.042857,737576.5625,645246.887324,645523.015152,679910.465116,728341.1875


In [ ]:
import plotly.express as px
fig = px.line(df_h5_m_prices.T)
fig.show()

NameError: name 'df_h5_m_prices' is not defined

In [ ]:
MoM = []

for i in range(1, 11):
    val1 = df_h5_m_prices.T.iloc[i-1]["mean"]
    val2 = df_h5_m_prices.T.iloc[i]["mean"]

    result = round(100*(val2 - val1)/val1,2)
    MoM.append(result)

print( f'A média de crescimento dos preços MoM foi de {np.mean(MoM):.3f}%' )

A média de crescimento dos preços MoM foi de 0.375%


#### **H6: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

#### **H1: Houses with waterfront are, on average, 30% more expensive.**

## 6.2. Answering Business Questions

### 1) Which are the real states that House Rocket should buy, and how much will it cost?

In [ ]:
house_rocket.columns.values

array(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'year',
       'month', 'season'], dtype=object)

In [56]:
df_table1 = house_rocket[['zipcode', 'price', 'condition']]
df_table1.head()

,zipcode,price,condition
0,98178,221900.00,3
1,98125,538000.00,3
2,98028,180000.00,3
3,98136,604000.00,5
4,98074,510000.00,3


In [54]:
df_region = house_rocket.groupby('zipcode').agg({'price': 'mean'})
df_region.head()

,price
zipcode,
98001,280804.69
98002,234284.04
98003,294111.28
98004,1355927.08
98005,810164.88


In [57]:
df_table1['Median Price'] = house_rocket.apply(lambda x: df_region.loc[x['zipcode']], axis=1)
df_table1.head()

/tmp/ipykernel_3542/574823041.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,zipcode,price,condition,Median Price
0,98178,221900.00,3,310612.76
1,98125,538000.00,3,469455.77
2,98028,180000.00,3,462480.04
3,98136,604000.00,5,551688.67
4,98074,510000.00,3,685605.78


In [65]:
df_region.loc[98028]

price   462480.04
Name: 98028, dtype: float64

In [68]:
df_table1['status'] = (df_table1['price'] < df_table1['Median Price']) & (df_table1['condition'] == 3)
print(df_table1.head())
print(df_table1['status'].unique())

   zipcode     price  condition  Median Price  status
0 98178.00 221900.00       3.00     310612.76    True
1 98125.00 538000.00       3.00     469455.77   False
2 98028.00 180000.00       3.00     462480.04    True
3 98136.00 604000.00       5.00     551688.67   False
4 98074.00 510000.00       3.00     685605.78    True
[ True False]


/tmp/ipykernel_3542/427497677.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 2) What is the best moment to sell the real states, and how much will it cost?

# 7) Solutions

# 8)  Conclusion


# 9)  Next Steps